In [1]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

postsDf = pd.read_csv("information_for_mongo.csv")

In [2]:
from typing import List
from together import Together
import json

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      info = f'"id": "{int(row["id"])}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|* "photo_caption": "{row["accessibility_caption"]}"'
      embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

      row_dict = {}
      for column in postsDf.columns.tolist():
        if column != "event_details":
          row_dict[column] = row[column]
      
      row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)
      row_dict["event_details"] = eval(row['event_details'])  
      
      result = collection.insert_one(row_dict)
      print(f"Inserted document ID: {result.inserted_id}")



        

Inserted document ID: 66a1dff9d5e43169157c9315
Inserted document ID: 66a1dffad5e43169157c9316
Inserted document ID: 66a1dffbd5e43169157c9317
Inserted document ID: 66a1dffbd5e43169157c9318
Inserted document ID: 66a1dffed5e43169157c9319
Inserted document ID: 66a1dffed5e43169157c931a
Inserted document ID: 66a1e002d5e43169157c931b
Inserted document ID: 66a1e002d5e43169157c931c
Inserted document ID: 66a1e005d5e43169157c931d
Inserted document ID: 66a1e006d5e43169157c931e
Inserted document ID: 66a1e009d5e43169157c931f
Inserted document ID: 66a1e009d5e43169157c9320
Inserted document ID: 66a1e00cd5e43169157c9321
Inserted document ID: 66a1e00dd5e43169157c9322
Inserted document ID: 66a1e010d5e43169157c9323


In [7]:

def return_events(prompt):
    query = prompt
    query_emb = generate_embedding([query], embedding_model_string)[0]

    results = collection.aggregate([
    {
        "$vectorSearch": {
        "queryVector": query_emb,
        "path": "embedded",
        "numCandidates": 100, # this should be 10-20x the limit
        "limit": 2, # the number of documents to return in the results
        "index": "vector_index", # the index name you used in Step 4.
        }
    }
    ])
    response = ""
    for i in results:
        response += (i['caption']+ "\n"+ i['url'] + '\n\n')
    return response

prompt = "Designing User Interfaces"

print(return_events(prompt))


We’re excited to announce our upcoming designathon: FRAME 🥳 

Here, you’ll design a solution for our given problem announced at the day of our event. Expect a day packed with new learnings, workshops, delicious food, making new friends, and more!

If you’re looking to connect with fellow creatives, industry professionals, and level up your design skills, this is YOUR SIGN to register and join! Students of all programs, backgrounds, and experience levels in design are welcome to join 🤩

When is this happening?
📅 Saturday, July 20th, 2024
⏰ 8:30 AM to 8:00 PM EST. doors open at 8:30!
👉 Location @ Communitech

🚨 DEADLINE to register for the event closed Tuesday, July 16th! 🚨

Update: SOLD OUT CAPACITY! We’ve sent an email to everyone who has signed up to fill out a MANDATORY form for finalized solo/team details ✨ This is DUE July 18th @ 11:59 PM EST ‼️

***

Haven’t attended a designathon before? Not to worry! We’ll have experienced mentors to support you through your design process and s